In [1]:
import sys
sys.path.append('../')
from delta_debugging.DD_mod import DDMods

## Example

In [2]:
class TestDD(DDMods):
	def __init__(self):
		DDMods.__init__(self)
		self.debug_dd = 0
		self.verbose = 0
		self.binary = False
	def _test(self, deltas):
		# Build input file
		found = []
		for (index, byte) in deltas:
			if byte == "<" or byte == ";" or byte == "1" or byte == "2":
				found.append(byte)
		ret = self.PASS
		if found.count("<") == 0 and found.count(";") == 0 and found.count("1") == 1 and found.count("2") == 3:
			ret = self.FAIL
		print('Testing case {:11}: {}'.format('"' + "".join([x[1] for x in deltas]) + '"', str(ret)))
		return ret

def str_to_deltas(test_input):
    deltas = list(map(lambda x: (x, test_input[x]), range(len(test_input))))
    return deltas

def deltas_to_str(deltas):
	return "".join([x[1] for x in deltas])

In [3]:
test_input = "1222>"
string2 = "<55513>;"
mydd = TestDD()
(c, c1, c2) = mydd.ddiff_max(test_input, string2)

Minimizing failure input: "1222>"
Testing case ""         : PASS
Testing case "1222>"    : FAIL
Testing case "22>"      : PASS
Testing case "12"       : PASS
Testing case "222>"     : PASS
Testing case "122>"     : PASS
Testing case "122>"     : PASS
Testing case "122"      : PASS
Testing case "122>"     : PASS
Testing case "1222"     : FAIL
Testing case "222"      : PASS
Testing case "122"      : PASS
Testing case "122"      : PASS
dd: done
Found minimal failure input: "1222"
Modifying failure input from  1222  towards  <55513>;
Testing case "<555122"  : PASS
Testing case "<51222"   : PASS
Testing case "551222"   : FAIL
Testing case "551"      : PASS
Testing case "<51"      : PASS
Testing case "<55512223>;": PASS
Testing case "<5551"    : PASS
Testing case "<551222"  : PASS
Testing case "5551222"  : FAIL
Testing case "5551"     : PASS
Testing case "<551"     : PASS
Testing case "<55512"   : PASS
Testing case "<55512"   : PASS
Testing case "<555123>;": PASS
The minimally different fail

## Step-by-step

In [4]:
mydd = TestDD()
test_input = "1222>"
print('Minimizing input: "{}"'.format(test_input))
# Convert string into the delta format
deltas = list(map(lambda x: (x, test_input[x]), range(len(test_input))))
c = mydd.ddmin(deltas)              # Invoke DDMIN
minimal = "".join([x[1] for x in c])
print('Found minimal test case: "{}"'.format(minimal))

Minimizing input: "1222>"
Testing case ""         : PASS
Testing case "1222>"    : FAIL
Testing case "22>"      : PASS
Testing case "12"       : PASS
Testing case "222>"     : PASS
Testing case "122>"     : PASS
Testing case "122>"     : PASS
Testing case "122"      : PASS
Testing case "122>"     : PASS
Testing case "1222"     : FAIL
Testing case "222"      : PASS
Testing case "122"      : PASS
Testing case "122"      : PASS
dd: done
Found minimal test case: "1222"


In [5]:
string1 = minimal
string2 = "<55513>;"
mods = mydd.get_mods(string1, string2)
c1 = str_to_deltas(string1)
c2 = str_to_deltas(string2)
c = mods
c

[(-4, '<', 'ADD'),
 (-3, '5', 'ADD'),
 (-2, '5', 'ADD'),
 (-1, '5', 'ADD'),
 (1, '2', 'REMOVE'),
 (2, '2', 'REMOVE'),
 (3, '2', 'REMOVE'),
 (3, '3', 'ADD'),
 (3, '>', 'ADD'),
 (3, ';', 'ADD')]

In [6]:
print("Expanding failure input ", mydd.pretty(c1), " to ", mydd.pretty(c2))

(c, c1, c2) = mydd.ddiff_mods(c1, c2, mods)
print("The minimally different failure to  ", mydd.pretty(c2), " is ", mydd.pretty(c1))
print("The difference is ", c)

Expanding failure input  1222  to  <55513>;
Testing case "<555122"  : PASS
Testing case "<51222"   : PASS
Testing case "551222"   : FAIL
Testing case "551"      : PASS
Testing case "<51"      : PASS
Testing case "<55512223>;": PASS
Testing case "<5551"    : PASS
Testing case "<551222"  : PASS
Testing case "5551222"  : FAIL
Testing case "5551"     : PASS
Testing case "<551"     : PASS
Testing case "<55512"   : PASS
Testing case "<55512"   : PASS
Testing case "<555123>;": PASS
The minimally different failure to   <55513>;  is  551222
The difference is  [(-4, '<', 'ADD'), (-3, '5', 'ADD'), (1, '2', 'REMOVE'), (2, '2', 'REMOVE'), (3, '2', 'REMOVE'), (3, '3', 'ADD'), (3, '>', 'ADD'), (3, ';', 'ADD')]
